In [1]:
!pip install Flask-Cors


In [ ]:
import io
import matplotlib.pyplot as plt
from flask import Flask, request, jsonify, Response
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import json
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from flask_cors import CORS
import os

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "http://localhost:4200"}})


def get_metrics(actual, predicted):
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    mae = np.mean(np.abs(actual - predicted))
    mse = np.mean((actual - predicted) ** 2)
    rmse = np.sqrt(mse)
    return {"MAPE": round(mape, 2), "MAE": round(mae, 2), "MSE": round(mse, 2), "RMSE": round(rmse, 2)}


@app.route('/', methods=['POST','GET'])
def forecast():
    file = request.files['csvFile']
    if file.filename == '':
        return "no file selected"
    file.save(file.filename)
    print('file saved')
    df = pd.read_csv(file.filename, parse_dates=['Order Date'], dayfirst=True)

    # get form-data parameters
    periodicity = request.form['periodicity']
    periods = int(request.form['periods'])

    # Extract the features and target variable
    df['year'] = df['Order Date'].dt.year
    df['month'] = df['Order Date'].dt.month
    df['week'] = df['Order Date'].dt.isocalendar().week
    df['day_of_week'] = df['Order Date'].dt.dayofweek
    df['day_of_month'] = df['Order Date'].dt.day
    df['lag_1'] = df['Sales'].shift(1).fillna(method='bfill')
    df['lag_2'] = df['Sales'].shift(2).fillna(method='bfill')
    X = df[['year', 'month', 'week', 'day_of_week', 'day_of_month', 'lag_1', 'lag_2']]
    y = df['Sales']

    # Train the random forest regressor model on the entire dataset
    rf_model = RandomForestRegressor()
    rf_model.fit(X, y)

    # Prepare new data for prediction (e.g., future dates)
    max_date = df['Order Date'].max()
    if periodicity == 'D':
        new_data = pd.date_range(start=max_date, periods=periods+1, freq=periodicity)[1:]
    else:
        new_data = pd.date_range(start=max_date, periods=periods, freq=periodicity)
    new_data_df = pd.DataFrame({'Order Date': new_data})
    new_data_df['year'] = new_data_df['Order Date'].dt.year
    new_data_df['month'] = new_data_df['Order Date'].dt.month
    new_data_df['week'] = new_data_df['Order Date'].dt.isocalendar().week
    new_data_df['day_of_week'] = new_data_df['Order Date'].dt.dayofweek
    new_data_df['day_of_month'] = new_data_df['Order Date'].dt.day
    new_data_df['lag_1'] = df['Sales'].iloc[-1]
    new_data_df['lag_2'] = df['Sales'].iloc[-2]
    new_data_X = new_data_df[['year', 'month', 'week', 'day_of_week', 'day_of_month', 'lag_1', 'lag_2']]
    # Make predictions on the new data
    new_data_predictions = rf_model.predict(new_data_X)

    # Get the actual values for the new data
    actual_values = y.iloc[-periods:].reset_index(drop=True)

    # Calculate the metrics for the predicted values
    metrics = get_metrics(actual_values, new_data_predictions)

    # Create a pandas dataframe to store the predictions
    predictions_df = pd.DataFrame({'Order Date': new_data, 'Sales Predictions': new_data_predictions})

  # Convert the dataframe to CSV format
    csv_output = predictions_df.to_csv(index=False)
    
    file_path = 'predictions.csv'
    with open(file_path, 'w') as f:
        f.write(csv_output)

     # check if the file was saved successfully
    if os.path.isfile(file_path):
        print('File saved successfully!')
    else:
        print('Error: File not saved.')
    

    # Set the response headers
    headers = {
        'Content-Disposition': 'attachment; filename="predictions.csv"',
        'Content-Type': 'text/csv'
    }

    # Return the response
    return Response(
        csv_output,
        headers=headers,
        mimetype='text/csv'
    ) 
if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


file saved


127.0.0.1 - - [03/Mar/2024 17:21:28] "POST / HTTP/1.1" 200 -


File saved successfully!
file saved


...

File saved successfully!


127.0.0.1 - - [03/Mar/2024 17:24:18] "POST / HTTP/1.1" 200 -


File saved successfully!
file saved


127.0.0.1 - - [03/Mar/2024 17:26:01] "POST / HTTP/1.1" 200 -


File saved successfully!
